# EDINET XBRL Analysis

設定を入力し、実行ボタンを押すだけで全自動でCSVを作成します。

In [ ]:
%load_ext autoreload
%autoreload 2
import edinet_xbrl_parser as parser

# =============================================================================
# 1. インプット設定
# =============================================================================

# XBRL Zipファイルがあるフォルダ
DIR_XBRL_ZIPS = r"XXX"

# 金融庁タクソノミ Zipファイルがあるフォルダ
DIR_TAXONOMY  = r"YYY"

# 企業ごとのマップJSONを保存するフォルダ (自動生成)
DIR_MAP_CACHE = r"company_maps"

# 抽出対象の企業 (パイプ区切りでOR検索。コード、社名など)
TARGET_COMPANIES = ['AAA|1234']

# 抽出対象の会計年度
TARGET_YEARS = [f"FY{y}" for y in range(2020, 2025)]

# 企業別辞書を強制的に作り直すか？ (通常は False でOK)
REBUILD_MAP = False

# =============================================================================
# 2. 実行
# =============================================================================
df_result = parser.execute_process(
    xbrl_zip_dir=DIR_XBRL_ZIPS,
    taxonomy_dir=DIR_TAXONOMY,
    map_cache_dir=DIR_MAP_CACHE,
    target_companies=TARGET_COMPANIES,
    target_years=TARGET_YEARS,
    rebuild_map=REBUILD_MAP
)

# Slide Generator

既存のdfを適当な形に整形のうえ、ぱわぽに埋め込みます。

In [ ]:
# ==============================================================================
# 0. dfの形式を変更（前処理として）
# ==============================================================================
df_for_pl_silde_TEMP = df_result[(df_result['証券コード']=='12345') & (df_result['カテゴリ']=='PL') & ~(df_result['単体連結区分']=='単体') & (df_result['詳細文脈']=='-') & (df_result['単位']=='JPY')]
df_pivoted = df_for_pl_silde_TEMP.pivot_table(
    index=['企業名', '会計年度', 'カテゴリ', '単位', '単体連結区分'], 
    columns='項目名(日本語)', 
    values='値',
    aggfunc='mean'
).reset_index()
arrX = [c for c in df_pivoted.columns if c not in ['企業名', '会計年度', 'カテゴリ', '単位', '単体連結区分']]
target_order = ['企業名', '会計年度', 'カテゴリ'] + arrX + ['単位', '単体連結区分']
df_for_pl_silde = df_pivoted[target_order]
df_for_pl_silde.columns.name = None

In [ ]:
# --- KPIの計算 ---
# 利益率を計算して新しいカラムに追加します。
# 事業利益率 (%)
df_for_pl_silde['事業利益率'] = (df_for_pl_silde['事業利益'] / df_for_pl_silde['売上高'] * 100).round(1)
# 純利益率 (%)
df_for_pl_silde['純利益率'] = (df_for_pl_silde['当期利益（△損失）'] / df_for_pl_silde['売上高'] * 100).round(1)

In [ ]:
# ==============================================================================
# 1. 初期設定とライブラリのインポート
# ==============================================================================
import pandas as pd
import os
import sys
from datetime import datetime
from slides_core import SlideConfig, PowerPointGeneratorEngine

# ==============================================================================
# 2. 環境設定 (Config Configuration)
# ==============================================================================

# --- ▼▼▼ ここはユーザ環境に合わせて更新してください ▼▼▼ ---

# 1. テンプレートファイルのパス (絶対パスでの指定を推奨)
# 例: "C:\\Users\\YourName\\Documents\\Templates\\template_v1.pptx"
# Notebookと同じフォルダにあるなら、ファイル名だけでOKです。
template_path = os.path.abspath("template.pptx") 

# 2. 生成されたパワポの出力先フォルダ
output_dir_base = os.path.abspath("out")
print(f"出力先ベースフォルダ（絶対パス）: {output_dir_base}")
timestamp = pd.Timestamp.now().strftime("%Y%m%d-%H%M%S")
output_dir = os.path.join(output_dir_base, f"result-{timestamp}")
print(f"今回の出力先フォルダ: {output_dir}")

# --- ▲▲▲ 更新ここまで ▲▲▲ ---

# 設定オブジェクトの生成
# (パスが見つからない場合は、この時点でエラーが出て教えてくれます)
try:
    config = SlideConfig(template_path=template_path, output_dir=output_dir)
    print("✅ 設定コンフィグの初期化に成功しました。")
    print(f"  - テンプレート: {config.paths.template_file}")
    print(f"  - 出力先: {config.paths.output_dir}")
except Exception as e:
    print("❌ 設定エラー:", e)
    raise e

# ==============================================================================
# 3. スライド構成とコンテンツの定義 (The Core Definition)
# ==============================================================================

# --- A. 表紙のコンテンツ ---
cover_content = {
    'main_title': '【ご提案資料】X社のPL分析',
    'sub_title': 'てすとだよ',
    'date': pd.Timestamp.now().strftime("%Y年%m月%d日") # 今日の日付を自動設定
}

# --- B. 各スライドの構成定義リスト ---
# ここに辞書を追加していくことで、スライドを何枚でも自動生成できます。
slides_structure = [
    # ==========================================================================
    # スライド1枚目
    # ==========================================================================
    {
        # スライド上部のタイトル
        'slide_title': '連結業績推移',
        
        # グラフの種類を指定（現在は 'COMBO_BAR_LINE_2AXIS' のみ対応）
        'category': 'COMBO_BAR_LINE_2AXIS',
        
        # データのマッピング定義（どのカラムをどう表示するか）
        'data_mapping': {
            'x_col': '会計年度', # X軸にするカラム名
            # 左軸（棒グラフ）の定義リスト
            'bar_traces': [
                {'col': '売上高', 'name': '売上高', 'color_key': 'navy'},  # col は df の中のカラム名、name はグラフ凡例、color_key はグラフの色
                # メインの利益は赤く目立たせる
                {'col': '事業利益', 'name': '事業利益', 'color_key': 'red', 'marker_size': 10, 'line_width': 3},
            ],
            # 右軸（折れ線グラフ）の定義リスト
            'line_traces': [
                # 利益率は準強調色で
                {'col': '事業利益率', 'name': '事業利益率', 'color_key': 'teal'}
            ]
        },
        
        # グラフ自体のテキスト要素
        'chart_text': {
            'title': '売上高・事業利益および事業利益率の推移',
            'y1_label': '売上高 & 事業利益 (円)', # 左軸ラベル
            'y2_label': '事業利益率 (%)'          # 右軸ラベル
        },
        
        # 右側の提案ポイント定義
        'proposal_section_title': '■ 業績トレンドからの示唆と評価', # セクションタイトル
        'proposal_points': [
            {
                'title': '考察1', 
                'body': 'がんばっているみたいです',
                'title_color_key': 'white'
            },
            {
                'title': '考察2', 
                'body': 'ここがだめみたいです',
                'accent_color_key': 'teal', # アクセントカラーを少し変えてみる
                'title_color_key': 'white'
            }
        ]
    },

    # ==========================================================================
    # スライド2枚目:
    # ==========================================================================
    {
        'slide_title': '次の例だよ',
        'category': 'COMBO_BAR_LINE_2AXIS', # 同じグラフ形式を再利用
        'data_mapping': {
            'x_col': '会計年度',
            'bar_traces': [
                # 棒グラフを2本並べて比較してみる
                {'col': '事業利益', 'name': '事業利益', 'color_key': 'navy'},
                {'col': '当期利益（△損失）', 'name': '当期利益（△損失）', 'color_key': 'teal'}
            ],
            'line_traces': [
                # ここでは純利益率にフォーカス
                {'col': '純利益率', 'name': '純利益率', 'color_key': 'red'}
            ]
        },
        'chart_text': {
            'title': '事業利益・当期純利益と純利益率の推移',
            'y1_label': '利益額 (円)',
            'y2_label': '純利益率 (%)'
        },
        'proposal_section_title': '■ いえること',
        'proposal_points': [
            {
                'title': '考察A', 
                'body': 'すごいねぇ',
                'title_color_key': 'white'
            },
            {
                'title': '考察B', 
                'body': 'ここもすごいねぇ',
                'accent_color_key': 'red', # ここは強く主張したいので赤にする
                'title_color_key': 'white'
            }
        ]
    }
    # 必要に応じて、ここにスライド定義の辞書を追加していけば何枚でも作れます。
]

print(f"✅ スライド構成の定義が完了しました。（合計 {len(slides_structure)} 枚のコンテンツスライド）")

# ==============================================================================
# 4. 実行 (Generate)
# ==============================================================================
print("🚀 PowerPoint生成エンジンを起動します...")

try:
    # エンジンのインスタンス化（Configを注入）
    engine = PowerPointGeneratorEngine(config)

    # 生成実行（データと定義を渡す）
    # filename_prefix は出力ファイル名の先頭につく文字列です。
    engine.generate(
        df=df_for_pl_silde,
        cover_content=cover_content,
        slides_structure=slides_structure,
        filename_prefix="Proposal_Draft_V2"
    )
    print("\n🎉 すべての処理が正常に終了しました。PowerPointを確認してください。")

except Exception as e:
    print("\n⛔ エラーが発生し、処理を中断しました。")
    # エラー内容を詳細に表示（デバッグ用）
    import traceback
    traceback.print_exc()